# V2 Notebook 4: The Robust MPC Core: Integrating Intelligence

**Project:** `RobustMPC-Pharma` (V2)
**Goal:** To assemble our advanced components into the final `RobustMPCController`. This new controller core will be uncertainty-aware, adaptive to disturbances, and capable of sophisticated optimization. This is where all our hard work comes together.

### Table of Contents
1. [Theory: The Pillars of a Robust Controller](#1.-Theory:-The-Pillars-of-a-Robust-Controller)
2. [Implementing the `RobustMPCController` V2](#2.-Implementing-the-RobustMPCController-V2)
3. [Standalone Test of the Integrated Controller](#3.-Standalone-Test-of-the-Integrated-Controller)

--- 
## 1. Theory: The Pillars of a Robust Controller

Our V2 controller is built on three pillars that address the weaknesses of the V1 prototype:

1.  **Stable State Perception (The Kalman Filter):** It will not react to raw sensor noise. Instead, it will act on a smooth, stable estimate of the true process state. This prevents control jitter and improves efficiency.

2.  **Risk-Aware Decision Making (The Probabilistic Model):** The controller's cost function will not just evaluate the model's mean prediction. It will incorporate the model's uncertainty (standard deviation). By optimizing a **risk-adjusted** forecast (e.g., Upper Confidence Bound), the controller will act more cautiously when the model is uncertain, making it fundamentally safer.

3.  **Adaptability to Disturbances (Integral Action):** Real-world processes suffer from un-modeled, persistent disturbances (e.g., gradual equipment fouling, changes in raw material). A standard controller will have a steady-state error in these cases. By implementing **Integral Action**, our controller will learn to recognize and automatically compensate for these disturbances, ensuring it always drives the process precisely to the target setpoint. This is known as **Offset-Free MPC**.

--- 
## 2. Implementing the `RobustMPCController` V2

We will now create the final `RobustMPCController` class in `src/core.py`. This class will be a high-level orchestrator, managing the interactions between the state estimator, the predictive model, and the optimizer.

The core logic will follow this sequence:
1.  Receive a noisy measurement from the plant.
2.  Use the `KalmanStateEstimator` to get a clean state estimate.
3.  Update its internal `disturbance_estimate` using the error between the setpoint and the clean state (Integral Action).
4.  Define a `fitness_function` for the optimizer. This function will:
    a. Take a candidate control plan.
    b. Get a probabilistic prediction from the `ProbabilisticTransformer`.
    c. Add the `disturbance_estimate` to the mean prediction.
    d. Calculate a risk-adjusted cost using the mean and standard deviation.
5.  Pass this fitness function to the `GeneticOptimizer` to find the best control plan.
6.  Return the first step of the winning plan.

--- 
## 3. Standalone Test of the Integrated Controller

Before the final showdown in Notebook 5, let's do a quick standalone test of the new `RobustMPCController` class. This will ensure all the components are communicating correctly. We will mock the necessary inputs and call the main `.suggest_action()` method to see if it produces a sensible result.

In [2]:
# This cell is for testing the class logic. A full implementation requires all components.
# Due to the complexity of mocking all inputs (history, scalers, etc.), a full test is deferred to Notebook 5.
# Here we will just instantiate the class to check for syntax errors.

from V2.robust_mpc.core import RobustMPCController
from V2.robust_mpc.estimators import KalmanStateEstimator
from V2.robust_mpc.models import ProbabilisticTransformer
from V2.robust_mpc.optimizers import GeneticOptimizer

# --- Mock Components and Configs (for instantiation) ---
mock_model = ProbabilisticTransformer(cma_features=2, cpp_features=5, d_model=32, nhead=2)
mock_estimator = 'KalmanFilterPlaceholder'
mock_optimizer_class = GeneticOptimizer
mock_scalers = 'ScalersPlaceholder'

MPC_CONFIG_V2 = {
    'lookback': 36,
    'horizon': 10, # Shorter for faster testing
    'cma_names': ['d50', 'lod'],
    'cpp_names': ['spray_rate', 'air_flow', 'carousel_speed'],
    'cpp_full_names': ['spray_rate', 'air_flow', 'carousel_speed', 'specific_energy', 'froude_number_proxy'],
    'integral_gain': 0.1,
    'risk_beta': 1.5,
    'mc_samples': 10,
    'cpp_constraints': {
        'spray_rate': {'min_val': 80.0, 'max_val': 180.0},
        'air_flow': {'min_val': 400.0, 'max_val': 700.0},
        'carousel_speed': {'min_val': 20.0, 'max_val': 40.0}
    },
    'ga_config': {
        'population_size': 40,
        'num_generations': 15,
        'crossover_prob': 0.7,
        'mutation_prob': 0.2,
        'horizon': 10, # Must match outer horizon
        'num_cpps': 3
    }
}

try:
    controller = RobustMPCController(
        model=mock_model,
        estimator=mock_estimator,
        optimizer_class=mock_optimizer_class,
        config=MPC_CONFIG_V2,
        scalers=mock_scalers
    )
    print("RobustMPCController class instantiated successfully!")
    print("Note: The helper methods for scaling and history need to be fully implemented for the final test.")
except Exception as e:
    print(f"An error occurred during instantiation: {e}")

RobustMPCController class instantiated successfully!
Note: The helper methods for scaling and history need to be fully implemented for the final test.


### Final Analysis and Next Steps

We have successfully designed and implemented the architecture for our advanced `RobustMPCController`. This new core brings together all the state-of-the-art components we've developed in this V2 series. While we have only performed a basic instantiation test here, we have laid out the complete logic that will be put to the test in our final notebook.

The key takeaways are:
*   **Modular Design:** The controller is built to accept different estimators, models, and optimizers, making it highly flexible.
*   **Integral Action:** The logic for `disturbance_estimate` is in place, ready to eliminate steady-state error.
*   **Risk-Awareness:** The fitness function is designed to use the probabilistic output of our model, making the controller inherently safer.

In the final notebook, we will implement the missing helper methods, connect this controller to our plant, and perform a head-to-head showdown against our V1 controller to definitively prove its superior performance.